# neural net to find object center, reducing failure rate

### Learning to optimize a Neural Net, part 2
This code goes with [this blogpost](https://medium.com/@mlrik/learning-to-optimize-a-neural-net-part-2-4397922d0243) (Draft)

This experiment is designed to measure the effects of reinitializing the learned weights of a NN. The second post in blog explores different ways to reduce the failure rate.

The failure rate was defined as "proportion of models with loss>0.45 after 2000 epochs of training from scratch".


<img src="PaperNet.jpg">

the initially designed NN has no convolutional layers. The Keras implementation looked like this:
```python
model.add(Dense(4, use_bias=True, activation=None, input_shape=(img_size,)))
model.add(Dense(2, use_bias=True, activation='relu'))
model.add(Dense(1, use_bias=False, activation=None))
```

In the previous [blogpost](https://medium.com/@mlrik/learning-to-optimize-a-neural-net-d60e85135b49), I proposed to myself to reduce the failure rate with one or more of the following:
1. activation functions (non-linearities) in all layers;
2. more layers;
3. more units per layer;
4. a bigger data set (add a bit of noise to the pixel values);
5. bigger images;
6. select the best models early in the process, clone them to maintain their numbers, add some randomness to the weights for variety, cull the others, train some more, repeat;

## init and import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import gc

%matplotlib inline

### a wee warning
This notebook keeps everything in memory. You might run into problems when increasing img_size or sprawl beyond a certain limit.
### a big warning
My code below is sloppy. Some functions rely on global variables to work. This is prototyping, no more, no less.

## create image functions

In [ ]:
def make_blob_images(img_size = 3, normx = False, normy = False):
    """create images (X) and labels (Y) indicating center of the object
    each image has an object and some empty space
    images have height of 1 pixel and width of img_size pixels"""
    
    X = []
    Y = []
    if normx:
        pixelvalue = 1
    else:
        pixelvalue = 2*img_size
    #
    for blob_size in range(1, img_size):
        for start in range(0, img_size - blob_size + 1):
            img = [0] * img_size
            img[start:start+blob_size] = [pixelvalue] * blob_size
            center = start + blob_size/2
            if normy:
                center = center / img_size
            X.append(img)
            Y.append(center)
        #
    #
    X = np.array(X)
    Y = np.array(Y)
    Y = Y.reshape(Y.shape[0], 1)
    return X, Y

In [ ]:
def plot_img(x,y_true, normy=False):
    """visualize an image (x) with the center of object (y_true) using matplotlib"""
    if normy:
        y_true = y_true * img_size
    img = x.reshape(1,img_size)
    plt.imshow(img, cmap='binary', extent=[0,img_size,0,1])
    plt.plot([y_true,y_true],[0,1], color='r', linewidth=2)
    plt.xticks(np.arange(0,img_size+1))
    plt.yticks([])
    plt.grid(axis='x')
    plt.show()

## create X and Y: images and blob center values
For a given size of the image. Display one such image.

In [ ]:
img_size = 3
X,Y = make_blob_images(img_size, normx=True, normy=True)

In [ ]:
i=0

In [ ]:
# execute this cell again and again to see more samples of X,Y
x = X[i]
y=Y[i]

plot_img(x,y, normy=True)
i = (i + 1) % len(X) # =0

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
print(np.mean(Y))
print(np.std(Y))
print(np.std(Y) ** 2)

In [ ]:
# plot all images in one plot
w = len(X[0])
h = len(X)
nominator = w * 2
nom = "{}th".format(nominator)
enums = []
for i in (Y*nominator).astype(int):
    enums.append("{}/{}".format(i[0], nom))
#

# plt.figure(figsize=(w/2,h/2))
plt.imshow(X, cmap='binary', extent=[0,w,0,h])
plt.grid(axis='both')

plt.xticks(np.arange(0,w),)
plt.yticks(np.arange(0,h+1), enums) # TODO hardcoded values into vars
plt.xlabel("pixel")
plt.ylabel("horizontal center")
plt.show()

### side note on the perfect outputs and the perfect loss
An object in 3-pixel-images has a horizontal center at exactly 1/6th of the width of the image. Or at a multiple of 1/6th. There are no images in this set with object width of zero or six.

The mean of all possible blob centers lies at exactly 0.500 (3/6th). The distance between two centers is always 0.1667 (1/6th). 

Any model that can predict all 5 centers within half that distance (*) of the true center, is performing "good enough". This is to say: a Mean Absolute Error below 0.0833 (1/12th) is a reasonable measure of a good model. However, a MAE<0.0833 does not promise if the model will predict correctly for all images. It is only an average error.

To call a model perfect, we would need to assess every single prediction for every possible image. And determine if they all stay below the calculated threshold.

In [ ]:
good_enough_loss = 1 / (img_size * 4)
print("good enough loss {:.4f}".format(good_enough_loss))

(*) Let's just ignore the edge cases where a blob lies at the edge of the image, where the model predicts its center beyond the width of the image and where we round that off to the nearest realistic value.

## create functions for model handling
Activate the make_model() function that you want to experiment with.

In [ ]:
# unchanged PaperNet version
def make_model(img_size=3, nn_depth=3, ll_width=3):
    """
    create a NN model according to the PaperNet architecture
    input_shape depends on img_size
    output_shape is 1
    """
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(4, use_bias=True, activation=None , input_shape=(img_size,) ) )
    model.add(keras.layers.Dense(2, use_bias=True, activation='relu' ) )
    model.add(keras.layers.Dense(1, use_bias=False, activation=None ) )
    model = compile_model(model)
    return model

In [ ]:
def clone_model(parent):
    """returns exact, compiled, copy of model with weights and all"""
    model = keras.models.clone_model(parent)
    W = parent.get_weights()
    model.set_weights(W)
    model = compile_model(model)
    return model

In [ ]:
def compile_model(model):
    """
    returns compiled PaperNet model with SGD optimizer and hard coded hyperparameters
    loss is defined as MAE (mean absolute error)
    """
    model.compile(optimizer=keras.optimizers.SGD(lr=.01), loss=keras.losses.mean_absolute_error, metrics=['mean_absolute_percentage_error'])
    return model

In [ ]:
def train(model, epochs = 100):
    """
    trains model using batch_size=len(X)
    returns training history object
    """
    history = model.fit(x=X, y=Y, epochs=epochs, batch_size=len(X), verbose=False)
    return history

In [ ]:
def evaluate(model):
    """evaluates model, returns loss and metrics"""
    return model.evaluate(x=X, y=Y, verbose=False)

In [ ]:
# dynamic version, model size changes as requested
def waxon_waxoff(parent=None, sprawl=1, epochs=100, nn_depth=3, ll_width=3): 
    """
    without parent, creates sprawl number of models, each with different random weights
    if parent model is given, will clone_model(parent) instead, resulting in sprawl identical models
    each model is trained a given number of epochs
    returns three lists with corresponding index: 
    children (contains model objects),
    stats (contains loss values),
    curves (contains history objects)
    """
    children = []
    stats = []
    curves = []
    
    print("train {} models, train each model for {} epochs".format(sprawl, epochs))
    for r in range(sprawl):
        print(" model:", r)
        if parent == None:
            model = make_model(img_size=img_size, nn_depth=nn_depth, ll_width=ll_width)
        else:
            model = clone_model(parent)
        #
        history = train(model=model, epochs = epochs)
        loss = evaluate(model=model)[0]

        children.append(model)
        stats.append(loss)
        curves.append(history)
    #
    return children, stats, curves



## create functions for reviewing experiments

In [ ]:
def report_experiment(version='baseline'):
    """
    report on an experiment with all the trimmings
    """
    print("Report for experiment titled \'{}\'".format(version))
    print("mean loss: {:.4f}  std dev loss: {:.4f}".format(np.mean(stats[version]), np.std(stats[version])) )
    print("best loss: {:.4f}    worst loss: {:.4f}".format(min(stats[version]), max(stats[version])) )
    print("best model: {}      worst model: {}".format(best[version], worst[version]))
    a=np.array(stats[version])
    b = a > 0.45
    print("failure rate for version {}: {}".format(version, b.sum() /len(b)))
    
    # plot predictions by Best model
    model = generation[version][best[version]]
    y_hat = model.predict(x=X).flatten()
    pos = range(len(Y.flatten()))
    plt.bar(pos, Y.flatten(), -.8, align='center', color='black', label='true')
    plt.bar(pos, y_hat, .3, align='edge', color='#3a9362', label='predict')
    plt.title("{} epochs per training run\npredictions by best of {} models".format(epochs[version], sprawl[version]))
    plt.xlabel("sample image")
    plt.ylabel("output")
    plt.legend()
    plt.show()

    # plot predictions by Worst model
    model = generation[version][worst[version]]
    y_hat = model.predict(x=X).flatten()
    pos = range(len(Y.flatten()))
    plt.bar(pos, Y.flatten(), -.8, align='center', color='black', label='true')
    plt.bar(pos, y_hat, .3, align='edge', color='#3a9362', label='predict')
    plt.title("{} epochs per training run\npredictions by worst of {} models".format(epochs[version], sprawl[version]))
    plt.xlabel("sample image")
    plt.ylabel("output")
    plt.legend()
    plt.show()

    # plot all losses for all models in the experiment
    pos = range(len(stats[version]))
    plt.bar(pos,stats[version])
    plt.title("losses in order, trained {} models for {} epochs".format(sprawl[version], epochs[version]))
    plt.xlabel("model")
    plt.ylabel("loss")
    plt.show()

    # plot histogram of losses of all models in the generation
    plt.hist(x=stats[version], bins=9)
    plt.title("losses histogram, trained {} models for {} epochs".format(sprawl[version], epochs[version]))
    plt.xlabel("loss")
    plt.ylabel("absolute freq.")
    plt.show()

    # plot learning curves for all models in one graph
    nicknames = range(len(curves[version]))
    plt.figure(figsize=(14,6))
    if len(nicknames) > 20:
        gray = True
    else:
        gray = False
    #
    for c in nicknames:
        h = curves[version][c].history['loss']
        if gray:
            plt.plot(h, label=c, color='gray', alpha=0.1)
        else:
            plt.plot(h, label=c)
        #
        plt.xscale("log")
    #
    plt.title("{} models trained for {} epochs\nexperiment titled {}".format(sprawl[version], epochs[version], version))
    if not gray:
        plt.legend()
    #
    plt.xlabel("epoch (log scale)")
    plt.ylabel("loss")
    plt.show()



In [ ]:
def summary_experiment(version='baseline'):
    """
    summary of an experiment, just the numbers
    returns failure_rate, best_loss
    """
    a=np.array(stats[version])
    b = a > 0.45
    failure_rate = b.sum() /len(b)
    best_loss = min(stats[version])

    return failure_rate, best_loss
    

In [ ]:
def summary_model(model):
    """
    summary of a model, just the numbers
    returns accuracy, error_max
    """
    # accuracy of model
    # is calculated as the proportion of images in X that the model predicts within good_enough margin
    # predict for the entire set X
    y_hat = model.predict(x=X)
    # make sure we work with flatten()ed arrays
    y = Y.flatten()
    y_hat = y_hat.flatten()
    # subtract
    errors = y - y_hat
    # get rid of minus signs
    absolute_errors = np.abs(errors)
    # create a binary map of the array absolute_errors (better vs worse than the norm)
    b = absolute_errors < good_enough_loss
    # count predictions that are good_enough, divide by total number of predictions
    accuracy = b.sum() / len(b)
    
    # what is the biggest error in the prediction
    error_max = np.max(absolute_errors)

    return accuracy, error_max
    

In [ ]:
def report_model(model):
    """
    report on a model with graphs and stats
    """
    
    # check model's architecture
    model.summary()
    
    # make a full prediction on the entire training set
    y_hat = model.predict(x=X).flatten()
    y = Y.flatten()

    # compute errors on all individual images and the mean absolute error
    e = y - y_hat
    ae = np.abs(e)
    mae=np.mean(ae)
    
    # prepare for the graph
    bot_margin = y - good_enough_loss
    margin = [good_enough_loss*2] * len(y)
    pos = range(len(y))

    # plot bar graph of ground truth, acceptable margins, predictions, errors
    plt.bar(pos, margin, .5, bot_margin, align='center', color='gray', alpha=.7)
    plt.bar(pos, y, -.8, align='center', color='black', label='true', alpha=.7)
    plt.bar(pos, y_hat, .3, align='edge', color='#3a9362', label='predict')
    plt.bar(pos, e, .1, y_hat, align='edge', color='orange', label='error')

    plt.title("version {} MAE={:.4f}".format(version, mae))
    plt.xlabel("sample image")
    plt.ylabel("output")
    plt.legend()
    plt.show()

## improve PaperNet failure rate: different approaches

In [ ]:
# from here on, we store results of experiments in these dicts:
generation = {}
stats = {}
curves = {}
best = {}
worst = {}
# experiment's parameters are also stored in dicts:
sprawl = {}
epochs = {}

### Set the baseline from the first notebook

In [ ]:
## name the experiment
version = 'papernet' #  a label to give the experiment

In [ ]:
## run the experiment, make sure to activate the correct cell with make_model()
sprawl[version] = 12
epochs[version] = 1400
generation[version], stats[version], curves[version] = waxon_waxoff(sprawl = sprawl[version], epochs=epochs[version] )
best[version]  = np.argmin(stats[version])
worst[version] = np.argmax(stats[version])


In [ ]:
report_experiment(version=version)

In [ ]:
report_model(generation[version][best[version]])

### This would be where you hack make_model() to do something different (or something similar, it's  up to you)

In [ ]:
# reset dicts:
generation = {}
stats = {}
curves = {}
best = {}
worst = {}
sprawl = {}
epochs = {}

gc.collect()

In [ ]:
## run the experiment
sprawl[version] = 13
epochs[version] = 3200
generation[version], stats[version], curves[version] = waxon_waxoff(sprawl = sprawl[version], epochs=epochs[version] )
best[version]  = np.argmin(stats[version])
worst[version] = np.argmax(stats[version])

In [ ]:
report_experiment(version='all_relu')

In [ ]:
# plot predictions by hand picked model
i=11
# i += 1 #=0

model = generation[version][i]
report_model(model)
summary_model(model)

In [ ]:
# choose the best model and clone, use the clone to parent more models
# m = clone_model(generation[version][best[version]])
m = clone_model(generation[version][11] )

In [ ]:
## run the experiment
sprawl[version] = 1
epochs[version] = 6000
generation[version], stats[version], curves[version] = waxon_waxoff(parent=m, sprawl = sprawl[version], epochs=epochs[version] )
best[version]  = np.argmin(stats[version])
worst[version] = np.argmax(stats[version])


In [ ]:
report_experiment(version='all_relu')

In [ ]:
model = generation[version][0]
report_model(model)
summary_model(model)

## Add more layers

In [ ]:
# try one extra layer, 4 layers in total
# img -> FC4 -> FC4 -> FC2 -> FC1 = output
version = 'base4'

In [ ]:
%time
## run the experiment, activate the corresponding make_model()
sprawl[version] = 40
epochs[version] = 800
generation[version], stats[version], curves[version] = waxon_waxoff(sprawl = sprawl[version], 
                                                                    epochs=epochs[version],
                                                                    nn_depth=4,
                                                                    ll_width=4,
                                                                   )
best[version]  = np.argmin(stats[version])
worst[version] = np.argmax(stats[version])

In [ ]:
report_experiment(version='base4')

In [ ]:
# choose the best model and clone, use the clone to parent more models
m = clone_model(generation[version][best[version]])

In [ ]:
## run the experiment
sprawl[version] = 1
epochs[version] = 100000
generation[version], stats[version], curves[version] = waxon_waxoff(parent=m, sprawl = sprawl[version], epochs=epochs[version] )
best[version]  = np.argmin(stats[version])
worst[version] = np.argmax(stats[version])

In [ ]:
report_experiment(version='base4')

In [ ]:
summary_experiment(version='base4')

### train models of varying depth

## More units per layer, starting by equalizing the number across layers

In [ ]:
# change make_model() again please
for nn_depth in [3,4,5,6,7,8,9]:
    version = "base_{}x4".format(nn_depth)
    
    ## run the experiment
    sprawl[version] = 11
    epochs[version] = 2200
    generation[version], stats[version], curves[version] = waxon_waxoff(sprawl = sprawl[version], 
                                                                        epochs=epochs[version],
                                                                        nn_depth=nn_depth,
                                                                        ll_width=4,
                                                                       )
    best[version]  = np.argmin(stats[version])
    worst[version] = np.argmax(stats[version])
    failure_rate, best_loss = summary_experiment(version=version)
    print("  {} layers in experiment {}, failure rate={:.4f}, best loss={:.4f}".format(nn_depth, version, failure_rate, best_loss))
    accuracy, error_max = summary_model(generation[version][best[version]])
    print("  best model ({}) has accuracy {:.4f} and a max error of {:.4f}".format(best[version], accuracy, error_max))
#


In [ ]:
print(generation.keys())

In [ ]:
topscores = []
worstpreds = []
losses = []
labels = ['base_3x4', 'base_4x4', 'base_5x4', 'base_6x4'] #, 'base_7x4', 'base_8x4', 'base_9x4']
for version in labels:
    loss = np.min(stats[version])
    accuracy, error_max = summary_model(generation[version][best[version]] )
    losses.append(loss)
    topscores.append(accuracy)
    worstpreds.append(error_max)
#
C = [1] * len(labels)
G = [good_enough_loss] * len(labels)
pos = range(len(labels))
plt.bar(pos, C, color='darkred')
plt.bar(pos, topscores, color='#3a9362')
plt.xticks(pos, labels)
plt.title('accuracy of best model in experiment')
plt.show()

plt.figure(figsize=(10,6))
plt.bar(pos, G, width=1, color='gray', alpha=.8, label='good enough')
plt.bar(pos, loss, width=1, color='blue', alpha=.4, label='model loss')
plt.bar(pos, worstpreds, width=.3, color='orange', alpha=.8, align='edge', label='loss on worst prediction')
plt.xticks(pos, labels)
plt.legend()
plt.title('worst prediction compared to model\'s overall loss')
plt.show()

In [ ]:
report_experiment(version='base_3x4')

In [ ]:
report_model(generation['base_7x4'][best['base_7x4']])

## Big Fat Experiment
Run 49 experiments to explore a 7x7 matrix of combo's.

In [ ]:
# reset dicts:
generation = {}
stats = {}
curves = {}
best = {}
worst = {}
sprawl = {}
epochs = {}

gc.collect()


In [ ]:
%%time
# change make_model() and waxon_waxoff() as necessary
big = [3,4,5,6,7,8,9]
small = [3,4]
beyondbig = [9,10,11,12,13]

depths = big
widths = big

for nn_depth in depths:
    for ll_width in widths:
        version = "base_{}l_{}u".format(nn_depth, ll_width)
        print("experiment", version)

        ## run the experiment
        sprawl[version] = 8
        epochs[version] = 1500
        generation[version], stats[version], curves[version] = waxon_waxoff(sprawl = sprawl[version], 
                                                                            epochs=epochs[version],
                                                                            nn_depth=nn_depth,
                                                                            ll_width=ll_width,
                                                                           )
        best[version]  = np.argmin(stats[version])
        worst[version] = np.argmax(stats[version])
    #
#
#  4 experiments in about 12 minutes
# 49 experiments in about 100 minutes
# 35 experiments in about 70 minutes

In [ ]:
d = len(depths)
w = len(widths)
xticks = range(min(widths), max(widths)+1)
yticks = range(min(depths), max(depths)+1)
y_offset = min(yticks)
x_offset = min(xticks)

em_map = np.zeros((d,w))
ac_map = np.zeros((d,w))
ls_map = np.zeros((d,w))

for nn_depth in depths:
    for ll_width in widths:
        version = "base_{}l_{}u".format(nn_depth, ll_width)

        if version in generation.keys():
            model = generation[version][best[version]]
            accuracy, error_max = summary_model(model)
            failure_rate, best_loss = summary_experiment(version=version)
            em_map[nn_depth-y_offset,ll_width-x_offset] = error_max
            ac_map[nn_depth-y_offset,ll_width-x_offset] = accuracy
            ls_map[nn_depth-y_offset,ll_width-x_offset] = best_loss           
        #
    #
#
plt.figure(figsize=(9,9))
plt.imshow(em_map,cmap='inferno')
plt.yticks(range(d),yticks)
plt.ylabel('depth')
plt.xlabel('width')
plt.xticks(range(w),xticks)
plt.colorbar(shrink=.8)
plt.title('error on hardest prediction by best model')
plt.show()

plt.figure(figsize=(9,9))
plt.imshow(ls_map,cmap='ocean')
plt.yticks(range(d),yticks)
plt.ylabel('depth')
plt.xlabel('width')
plt.xticks(range(w),xticks)
plt.colorbar(shrink=.8)
plt.title('loss overall of best model')
plt.show()

plt.figure(figsize=(9,9))
plt.imshow(ac_map,cmap='YlGn')
plt.yticks(range(d),yticks)
plt.xticks(range(w),xticks)
plt.ylabel('depth')
plt.xlabel('width')
plt.colorbar(shrink=.8)
plt.title('accuracy of best model')
plt.show()

In [ ]:
v = version = 'base_7l_12u'
m = generation[v][best[v]]
report_model(m )
print(v, m)


In [ ]:
summary_model(m)

In [ ]:
summary_experiment(version=version)

In [ ]:
report_experiment(version=version)

In [ ]:
h = curves[version]

In [ ]:
l=[]
for i in h:
    l.append(min(i.history['loss']))
print(min(l))

In [ ]:
print(min(l) * 15)

## take a look at the activations inside a model

In [ ]:
version = 'all_relu'

In [ ]:
# create three model clones with decreasing number of layers
model3 = generation[version][worst[version]]
W = model3.get_weights()

model2 = keras.models.clone_model(model3)
model2.set_weights(W)
model2.pop()

model1 = keras.models.clone_model(model3)
model1.set_weights(W)
model1.pop()
model1.pop()

In [ ]:
pred1 = model1.predict(x=X)
print(pred1)
plt.imshow(pred1, cmap='RdBu')
plt.colorbar()
plt.show()

In [ ]:
p = pred1.flatten()
plt.hist(p)
plt.show()

In [ ]:
pred2 = model2.predict(x=X)
print(pred2)
plt.imshow(pred2, cmap='RdBu')
plt.colorbar()
plt.show()

In [ ]:
p = pred2.flatten()
plt.hist(p)
plt.show()

In [ ]:
pred3 = model3.predict(x=X)
# print(pred3)
print(pred3)
plt.imshow(pred3, cmap='RdBu')
plt.colorbar()
plt.show()

In [ ]:
p = pred3.flatten()
plt.hist(p)
plt.show()

In [ ]:
# plot predictions by selected model
y_hat = model3.predict(x=X).flatten()
pos = range(len(Y.flatten()))
plt.bar(pos, Y.flatten(), -.8, align='center', color='black', label='true')
plt.bar(pos, y_hat, .3, align='edge', color='#3a9362', label='predict')
plt.title("experiment titled {}".format(version))
plt.xlabel("sample image")
plt.ylabel("output")
plt.legend()
plt.show()